You are working for an airline company looking to enter the United States domestic market.
Specifically, the company has decided to start with 5 round trip routes between medium and
large US airports. An example of a round trip route is the combination of JFK to ORD and ORD
to JFK. The airline company has to acquire 5 new airplanes (one per round trip route) and the
upfront cost for each airplane is $90 million. The company’s motto is “On time, for you”, so
punctuality is a big part of its brand image.
You have been tasked with analyzing 1Q2019 data to identify:

1. The 10 busiest round trip routes in terms of number of round trip flights in the quarter.
Exclude canceled flights when performing the calculation.<br><br>

2. The 10 most profitable round trip routes (without considering the upfront airplane cost) in
the quarter. Along with the profit, show total revenue, total cost, summary values of
other key components and total round trip flights in the quarter for the top 10 most
profitable routes. Exclude canceled flights from these calculations.<br><br>

3. The 5 round trip routes that you recommend to invest in based on any factors that you
choose.<br><br>

4. The number of round trip flights it will take to breakeven on the upfront airplane cost for
each of the 5 round trip routes that you recommend. Print key summary components for
these routes.<br><br>

5. Key Performance Indicators (KPI’s) that you recommend tracking in the future to
measure the success of the round trip routes that you recommend.<br><br>

In [30]:
import pandas as pd
import numpy as np

# Sample dataset
data = pd.DataFrame({
    'DEP': ['LA', 'LA', 'NY', 'KC', 'MD'],
    'ARR': ['NY', 'NY', 'LA', 'CHI', 'MIA'],
    'Passengers': [10, 15, 20, 25, 30]
})

df = pd.DataFrame(data)
df['route'] = df.apply(lambda row: '-'.join(sorted([row['DEP'], row['ARR']])), axis=1)

df_grouped = df.groupby(['route']).agg({'Passengers': np.sum}).reset_index()
                                       
#df_grouped = df_grouped.to_frame().reset_index() 
df_grouped

,route,Passengers
0,CHI-KC,25
1,LA-NY,45
2,MD-MIA,30


In [2]:
import pandas as pd
import numpy as np

In [8]:
file_path = '/Users/jean_taylor_1/Downloads/data/'

## 1.0 - Understand data sources provided

## 1.1 - Understand Airport Codes.csv<h2>

In [9]:
airport_codes = pd.read_csv(file_path + 'Airport_Codes.csv')

In [10]:
airport_codes.head(10)

,TYPE,NAME,ELEVATION_FT,CONTINENT,ISO_COUNTRY,MUNICIPALITY,IATA_CODE,COORDINATES
0,heliport,Total Rf Heliport,11.0,NaN,US,Bensalem,NaN,"-74.93360137939453, 40.07080078125"
1,small_airport,Aero B Ranch Airport,3435.0,NaN,US,Leoti,NaN,"-101.473911, 38.704022"
2,small_airport,Lowell Field,450.0,NaN,US,Anchor Point,NaN,"-151.695999146, 59.94919968"
3,small_airport,Epps Airpark,820.0,NaN,US,Harvest,NaN,"-86.77030181884766, 34.86479949951172"
4,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,Newport,NaN,"-91.254898, 35.6087"
5,small_airport,Fulton Airport,1100.0,NaN,US,Alex,NaN,"-97.8180194, 34.9428028"
6,small_airport,Cordes Airport,3810.0,NaN,US,Cordes,NaN,"-112.16500091552734, 34.305599212646484"
7,small_airport,Goldstone /Gts/ Airport,3038.0,NaN,US,Barstow,NaN,"-116.888000488, 35.350498199499995"
8,small_airport,Williams Ag Airport,87.0,NaN,US,Biggs,NaN,"-121.763427, 39.427188"
9,heliport,Kitchen Creek Helibase Heliport,3350.0,NaN,US,Pine Valley,NaN,"-116.4597417, 32.7273736"


In [11]:
airport_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55369 entries, 0 to 55368
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TYPE          55369 non-null  object 
 1   NAME          55369 non-null  object 
 2   ELEVATION_FT  48354 non-null  float64
 3   CONTINENT     27526 non-null  object 
 4   ISO_COUNTRY   55122 non-null  object 
 5   MUNICIPALITY  49663 non-null  object 
 6   IATA_CODE     9182 non-null   object 
 7   COORDINATES   55369 non-null  object 
dtypes: float64(1), object(7)
memory usage: 3.4+ MB


## 1.2 - Understand Flights.csv

In [74]:
flights = pd.read_csv(file_path + 'Flights.csv')

/var/folders/9b/4_3pqpvx3fs3ccpkbgvs21n40000gp/T/ipykernel_1051/3083784813.py:1: DtypeWarning: Columns (3,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  flights = pd.read_csv(file_path + 'Flights.csv')


In [31]:
flights.head(100)

,FL_DATE,OP_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DESTINATION,DEST_CITY_NAME,DEP_DELAY,ARR_DELAY,CANCELLED,AIR_TIME,DISTANCE,OCCUPANCY_RATE
0,2019-03-02,WN,N955WN,4591,14635,RSW,"Fort Myers, FL",11042,CLE,"Cleveland, OH",-8.0,-6.0,0.0,143.0,1025.0,0.97
1,2019-03-02,WN,N8686A,3231,14635,RSW,"Fort Myers, FL",11066,CMH,"Columbus, OH",1.0,5.0,0.0,135.0,930.0,0.55
2,2019-03-02,WN,N201LV,3383,14635,RSW,"Fort Myers, FL",11066,CMH,"Columbus, OH",0.0,4.0,0.0,132.0,930.0,0.91
3,2019-03-02,WN,N413WN,5498,14635,RSW,"Fort Myers, FL",11066,CMH,"Columbus, OH",11.0,14.0,0.0,136.0,930.0,0.67
4,2019-03-02,WN,N7832A,6933,14635,RSW,"Fort Myers, FL",11259,DAL,"Dallas, TX",0.0,-17.0,0.0,151.0,1005.0,0.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2019-03-02,WN,N8652B,2516,14679,SAN,"San Diego, CA",14893,SMF,"Sacramento, CA",-5.0,-10.0,0.0,76.0,480.0,0.36
96,2019-03-02,WN,N8671D,2789,14679,SAN,"San Diego, CA",14893,SMF,"Sacramento, CA",35.0,32.0,0.0,74.0,480.0,0.70
97,2019-03-02,WN,N966WN,3468,14679,SAN,"San Diego, CA",14893,SMF,"Sacramento, CA",2.0,0.0,0.0,72.0,480.0,0.87
98,2019-03-02,WN,N906WN,3544,14679,SAN,"San Diego, CA",14893,SMF,"Sacramento, CA",-1.0,-11.0,0.0,69.0,480.0,0.44


In [59]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1915886 entries, 0 to 1915885
Data columns (total 16 columns):
 #   Column             Dtype  
---  ------             -----  
 0   FL_DATE            object 
 1   OP_CARRIER         object 
 2   TAIL_NUM           object 
 3   OP_CARRIER_FL_NUM  object 
 4   ORIGIN_AIRPORT_ID  int64  
 5   ORIGIN             object 
 6   ORIGIN_CITY_NAME   object 
 7   DEST_AIRPORT_ID    int64  
 8   DESTINATION        object 
 9   DEST_CITY_NAME     object 
 10  DEP_DELAY          float64
 11  ARR_DELAY          float64
 12  CANCELLED          float64
 13  AIR_TIME           object 
 14  DISTANCE           object 
 15  OCCUPANCY_RATE     float64
dtypes: float64(4), int64(2), object(10)
memory usage: 233.9+ MB


In [60]:
flights.isnull().sum()

FL_DATE                  0
OP_CARRIER               0
TAIL_NUM             12156
OP_CARRIER_FL_NUM        0
ORIGIN_AIRPORT_ID        0
ORIGIN                   0
ORIGIN_CITY_NAME         0
DEST_AIRPORT_ID          0
DESTINATION              0
DEST_CITY_NAME           0
DEP_DELAY            50351
ARR_DELAY            55991
CANCELLED                0
AIR_TIME             56551
DISTANCE               630
OCCUPANCY_RATE         310
dtype: int64

## 1.2 - Understand Tickets.csv

In [16]:
tickets = pd.read_csv(file_path + 'Tickets.csv')

In [17]:
tickets.head(10)

,ITIN_ID,YEAR,QUARTER,ORIGIN,ORIGIN_COUNTRY,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,ROUNDTRIP,REPORTING_CARRIER,PASSENGERS,ITIN_FARE,DESTINATION
0,201912723049,2019,1,ABI,US,TX,Texas,1.0,MQ,1.0,736.0,DAB
1,201912723085,2019,1,ABI,US,TX,Texas,1.0,MQ,1.0,570.0,COS
2,201912723491,2019,1,ABI,US,TX,Texas,1.0,MQ,1.0,564.0,MCO
3,201912723428,2019,1,ABI,US,TX,Texas,1.0,MQ,1.0,345.0,LGA
4,201912723509,2019,1,ABI,US,TX,Texas,0.0,MQ,1.0,309.0,MGM
5,201912723565,2019,1,ABI,US,TX,Texas,0.0,MQ,2.0,303.0,MSY
6,201912723569,2019,1,ABI,US,TX,Texas,0.0,MQ,1.0,377.0,MSY
7,201912723350,2019,1,ABI,US,TX,Texas,0.0,MQ,1.0,339.0,LAS
8,201912723755,2019,1,ABI,US,TX,Texas,0.0,MQ,1.0,369.0,RNO
9,201912723645,2019,1,ABI,US,TX,Texas,0.0,MQ,1.0,369.0,ORF


In [18]:
tickets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167285 entries, 0 to 1167284
Data columns (total 12 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   ITIN_ID            1167285 non-null  int64  
 1   YEAR               1167285 non-null  int64  
 2   QUARTER            1167285 non-null  int64  
 3   ORIGIN             1167285 non-null  object 
 4   ORIGIN_COUNTRY     1167285 non-null  object 
 5   ORIGIN_STATE_ABR   1167285 non-null  object 
 6   ORIGIN_STATE_NM    1167285 non-null  object 
 7   ROUNDTRIP          1167285 non-null  float64
 8   REPORTING_CARRIER  1167285 non-null  object 
 9   PASSENGERS         1165308 non-null  float64
 10  ITIN_FARE          1166325 non-null  object 
 11  DESTINATION        1167285 non-null  object 
dtypes: float64(2), int64(3), object(7)
memory usage: 106.9+ MB


## 2.0 - Task 1
***The 10 busiest round trip routes in terms of number of round trip flights in the quarter. Exclude canceled flights when performing the calculation.***<br><br>
This task can be answered by the `Flights` and `Airport Codes` data.<br> 
Since the airline is interested in medium and large airports, we will merge those two datasets and
We are asked for the # of round trip flights, with 'round trip' defined in the instructions as a trip that departs and returns to the same airport (*Example being JFK to ORD and ORD to JFK as one round trip*).
There is no mention of including information about tickets that are round trip itineraries, so `Tickets` data is not needed.<br>
Althought it is common for a round trip ticket to be with the same airline, the task did not mention to call that out. Therefore, the distinct operating airlines were not included.

In [75]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1915886 entries, 0 to 1915885
Data columns (total 16 columns):
 #   Column             Dtype  
---  ------             -----  
 0   FL_DATE            object 
 1   OP_CARRIER         object 
 2   TAIL_NUM           object 
 3   OP_CARRIER_FL_NUM  object 
 4   ORIGIN_AIRPORT_ID  int64  
 5   ORIGIN             object 
 6   ORIGIN_CITY_NAME   object 
 7   DEST_AIRPORT_ID    int64  
 8   DESTINATION        object 
 9   DEST_CITY_NAME     object 
 10  DEP_DELAY          float64
 11  ARR_DELAY          float64
 12  CANCELLED          float64
 13  AIR_TIME           object 
 14  DISTANCE           object 
 15  OCCUPANCY_RATE     float64
dtypes: float64(4), int64(2), object(10)
memory usage: 233.9+ MB


In [247]:
# Retain info we need
flights1 = flights[['FL_DATE', 'ORIGIN', 'DESTINATION', 'CANCELLED']]

In [248]:
flights1

,FL_DATE,ORIGIN,DESTINATION,CANCELLED
0,2019-03-02,RSW,CLE,0.0
1,2019-03-02,RSW,CMH,0.0
2,2019-03-02,RSW,CMH,0.0
3,2019-03-02,RSW,CMH,0.0
4,2019-03-02,RSW,DAL,0.0
...,...,...,...,...
1915881,3/23/19,TUL,CLT,0.0
1915882,3/24/19,TUL,CLT,0.0
1915883,3/25/19,TUL,CLT,0.0
1915884,3/26/19,TUL,CLT,0.0


In [249]:
# I see above there is a date formating issue. Lets look into these date values
flights1['FL_DATE'].unique()

array(['2019-03-02', '2019-03-03', '2019-03-09', '2019-03-10',
       '2019-03-24', '2019-03-28', '2019-03-29', '2019-03-30',
       '2019-03-20', '2019-03-25', '2019-03-26', '2019-03-31',
       '2019-03-13', '2019-03-14', '2019-03-12', '2019-03-11',
       '2019-03-15', '2019-03-16', '2019-03-17', '2019-03-18',
       '2019-03-19', '2019-03-21', '2019-03-22', '2019-03-23',
       '2019-03-27', '2019-03-01', '2019-03-04', '2019-03-05',
       '2019-03-06', '2019-03-07', '2019-03-08', '2019-02-17',
       '2019-02-18', '2019-02-19', '2019-02-20', '2019-02-21',
       '2019-02-22', '2019-02-23', '2019-02-24', '2019-02-25',
       '2019-02-26', '2019-02-27', '2019-02-28', '2019-02-01',
       '2019-02-02', '2019-02-03', '2019-02-04', '2019-02-05',
       '2019-02-06', '2019-02-07', '2019-02-08', '2019-02-09',
       '2019-02-10', '2019-02-11', '2019-02-12', '2019-02-13',
       '2019-02-14', '2019-02-15', '2019-02-16', '2019-01-21',
       '2019-01-24', '2019-01-25', '2019-01-26', '2019-

In [250]:
flights1['FL_DATE'] = pd.to_datetime(flights1['FL_DATE'])

/var/folders/9b/4_3pqpvx3fs3ccpkbgvs21n40000gp/T/ipykernel_1051/2110118796.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights1['FL_DATE'] = pd.to_datetime(flights1['FL_DATE'])


In [251]:
flights1['FL_DATE'].unique()

array(['2019-03-02T00:00:00.000000000', '2019-03-03T00:00:00.000000000',
       '2019-03-09T00:00:00.000000000', '2019-03-10T00:00:00.000000000',
       '2019-03-24T00:00:00.000000000', '2019-03-28T00:00:00.000000000',
       '2019-03-29T00:00:00.000000000', '2019-03-30T00:00:00.000000000',
       '2019-03-20T00:00:00.000000000', '2019-03-25T00:00:00.000000000',
       '2019-03-26T00:00:00.000000000', '2019-03-31T00:00:00.000000000',
       '2019-03-13T00:00:00.000000000', '2019-03-14T00:00:00.000000000',
       '2019-03-12T00:00:00.000000000', '2019-03-11T00:00:00.000000000',
       '2019-03-15T00:00:00.000000000', '2019-03-16T00:00:00.000000000',
       '2019-03-17T00:00:00.000000000', '2019-03-18T00:00:00.000000000',
       '2019-03-19T00:00:00.000000000', '2019-03-21T00:00:00.000000000',
       '2019-03-22T00:00:00.000000000', '2019-03-23T00:00:00.000000000',
       '2019-03-27T00:00:00.000000000', '2019-03-01T00:00:00.000000000',
       '2019-03-04T00:00:00.000000000', '2019-03-05

In [252]:
flights1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1915886 entries, 0 to 1915885
Data columns (total 4 columns):
 #   Column       Dtype         
---  ------       -----         
 0   FL_DATE      datetime64[ns]
 1   ORIGIN       object        
 2   DESTINATION  object        
 3   CANCELLED    float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 58.5+ MB


In [253]:
# Convert airport 3 digit code data type from object to category for memory usage
flights1['ORIGIN'] = flights1['ORIGIN'].astype('category')
flights1['DESTINATION'] = flights1['DESTINATION'].astype('category')

# Convert cancelled True/False from int to bool
flights1['CANCELLED'] = flights1['CANCELLED'].astype('bool')

/var/folders/9b/4_3pqpvx3fs3ccpkbgvs21n40000gp/T/ipykernel_1051/3880754850.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights1['ORIGIN'] = flights1['ORIGIN'].astype('category')
/var/folders/9b/4_3pqpvx3fs3ccpkbgvs21n40000gp/T/ipykernel_1051/3880754850.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flights1['DESTINATION'] = flights1['DESTINATION'].astype('category')
/var/folders/9b/4_3pqpvx3fs3ccpkbgvs21n40000gp/T/ipykernel_1051/3880754850.py:4: SettingWithCopyWarning: 
A value is trying to be 

In [254]:
flights1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1915886 entries, 0 to 1915885
Data columns (total 4 columns):
 #   Column       Dtype         
---  ------       -----         
 0   FL_DATE      datetime64[ns]
 1   ORIGIN       category      
 2   DESTINATION  category      
 3   CANCELLED    bool          
dtypes: bool(1), category(2), datetime64[ns](1)
memory usage: 23.8 MB


In [255]:
flights1.head(100)

,FL_DATE,ORIGIN,DESTINATION,CANCELLED
0,2019-03-02,RSW,CLE,False
1,2019-03-02,RSW,CMH,False
2,2019-03-02,RSW,CMH,False
3,2019-03-02,RSW,CMH,False
4,2019-03-02,RSW,DAL,False
...,...,...,...,...
95,2019-03-02,SAN,SMF,False
96,2019-03-02,SAN,SMF,False
97,2019-03-02,SAN,SMF,False
98,2019-03-02,SAN,SMF,False


In [256]:
flights1 = flights1[flights1['CANCELLED'] == False]

In [257]:
flights1['CANCELLED'].unique()

array([False])

In [258]:
flights1 = flights1.drop('CANCELLED', axis=1).reset_index(drop=True)

In [259]:
flights1

,FL_DATE,ORIGIN,DESTINATION
0,2019-03-02,RSW,CLE
1,2019-03-02,RSW,CMH
2,2019-03-02,RSW,CMH
3,2019-03-02,RSW,CMH
4,2019-03-02,RSW,DAL
...,...,...,...
1864267,2019-03-23,TUL,CLT
1864268,2019-03-24,TUL,CLT
1864269,2019-03-25,TUL,CLT
1864270,2019-03-26,TUL,CLT


In [260]:
flights1.isnull().sum()

FL_DATE        0
ORIGIN         0
DESTINATION    0
dtype: int64

In [261]:
print(len(flights1))

1864272


Merge Airport codes data to include airport size

In [262]:
airport_codes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55369 entries, 0 to 55368
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TYPE          55369 non-null  object 
 1   NAME          55369 non-null  object 
 2   ELEVATION_FT  48354 non-null  float64
 3   CONTINENT     27526 non-null  object 
 4   ISO_COUNTRY   55122 non-null  object 
 5   MUNICIPALITY  49663 non-null  object 
 6   IATA_CODE     9182 non-null   object 
 7   COORDINATES   55369 non-null  object 
dtypes: float64(1), object(7)
memory usage: 3.4+ MB


In [263]:
del(flights1_airpot_code_merged)

In [264]:
flights1_airpot_code_merged = flights1.join(airport_codes.set_index('IATA_CODE')['TYPE'], on='ORIGIN')\
                                                                .rename(columns={'TYPE': 'ORIGIN_TYPE'})

flights1_airpot_code_merged = flights1_airpot_code_merged.join(airport_codes.set_index('IATA_CODE')['TYPE'], on='DESTINATION')\
                                                                .rename(columns={'TYPE': 'DESTINATION_TYPE'})

In [265]:
flights1_airpot_code_merged

,FL_DATE,ORIGIN,DESTINATION,ORIGIN_TYPE,DESTINATION_TYPE
0,2019-03-02,RSW,CLE,large_airport,large_airport
1,2019-03-02,RSW,CMH,large_airport,large_airport
2,2019-03-02,RSW,CMH,large_airport,large_airport
3,2019-03-02,RSW,CMH,large_airport,large_airport
4,2019-03-02,RSW,DAL,large_airport,large_airport
...,...,...,...,...,...
1864267,2019-03-23,TUL,CLT,large_airport,large_airport
1864268,2019-03-24,TUL,CLT,large_airport,large_airport
1864269,2019-03-25,TUL,CLT,large_airport,large_airport
1864270,2019-03-26,TUL,CLT,large_airport,large_airport


In [274]:
flights1_airpot_code_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1894498 entries, 0 to 1864271
Data columns (total 5 columns):
 #   Column            Dtype         
---  ------            -----         
 0   FL_DATE           datetime64[ns]
 1   ORIGIN            category      
 2   DESTINATION       category      
 3   ORIGIN_TYPE       category      
 4   DESTINATION_TYPE  category      
dtypes: category(4), datetime64[ns](1)
memory usage: 39.8 MB


In [272]:
flights1_airpot_code_merged['ORIGIN'] = flights1_airpot_code_merged['ORIGIN'].astype('category')
flights1_airpot_code_merged['DESTINATION'] = flights1_airpot_code_merged['DESTINATION'].astype('category')
flights1_airpot_code_merged['ORIGIN_TYPE'] = flights1_airpot_code_merged['ORIGIN_TYPE'].astype('category')
flights1_airpot_code_merged['DESTINATION_TYPE'] = flights1_airpot_code_merged['DESTINATION_TYPE'].astype('category')

In [273]:
flights1_airpot_code_merged['ORIGIN_TYPE'].value_counts()

large_airport     1702572
medium_airport     169523
closed              15098
small_airport        7305
Name: ORIGIN_TYPE, dtype: int64

In [290]:
flights1_airpot_code_merged = flights1_airpot_code_merged[flights1_airpot_code_merged['ORIGIN_TYPE'].isin(['large_airport', 'medium_airport'])]
flights1_airpot_code_merged = flights1_airpot_code_merged[flights1_airpot_code_merged['DESTINATION_TYPE'].isin(['large_airport', 'medium_airport'])].reset_index(drop=True)

In [291]:
flights1_airpot_code_merged['ORIGIN_TYPE'].value_counts()

large_airport     1681383
medium_airport     168349
closed                  0
small_airport           0
Name: ORIGIN_TYPE, dtype: int64

In [292]:
flights1_airpot_code_merged['DESTINATION_TYPE'].value_counts()

large_airport     1680688
medium_airport     169044
closed                  0
small_airport           0
Name: DESTINATION_TYPE, dtype: int64

In [293]:
flights1_airpot_code_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1849732 entries, 0 to 1849731
Data columns (total 5 columns):
 #   Column            Dtype         
---  ------            -----         
 0   FL_DATE           datetime64[ns]
 1   ORIGIN            category      
 2   DESTINATION       category      
 3   ORIGIN_TYPE       category      
 4   DESTINATION_TYPE  category      
dtypes: category(4), datetime64[ns](1)
memory usage: 24.7 MB


In [294]:
flights1_airpot_code_merged

,FL_DATE,ORIGIN,DESTINATION,ORIGIN_TYPE,DESTINATION_TYPE
0,2019-03-02,RSW,CLE,large_airport,large_airport
1,2019-03-02,RSW,CMH,large_airport,large_airport
2,2019-03-02,RSW,CMH,large_airport,large_airport
3,2019-03-02,RSW,CMH,large_airport,large_airport
4,2019-03-02,RSW,DAL,large_airport,large_airport
...,...,...,...,...,...
1849727,2019-03-23,TUL,CLT,large_airport,large_airport
1849728,2019-03-24,TUL,CLT,large_airport,large_airport
1849729,2019-03-25,TUL,CLT,large_airport,large_airport
1849730,2019-03-26,TUL,CLT,large_airport,large_airport


In [298]:
flights1_airpot_code_merged['ROUND_TRIP'] = flights1_airpot_code_merged.apply(lambda row: '-'.join(sorted([row['DESTINATION'], row['ORIGIN']])), axis=1)

flights1_airpot_code_merged.groupby(['ROUND_TRIP']).agg({'FL_DATE': 'count'}).sort_values('FL_DATE', ascending=False).head(10)
                                       


,FL_DATE
ROUND_TRIP,
LAX-SFO,8340
LGA-ORD,7156
LAS-LAX,6511
JFK-LAX,6320
LAX-SEA,4999
BOS-LGA,4820
HNL-OGG,4794
PDX-SEA,4774
ATL-MCO,4707


In [ ]:
flights1agg = flight1.groupby(['ORIGIN','DESTINATION']).agg


flights=flights.groupby(['ORIGIN','DESTINATION']).agg({'ORIGIN_CITY_NAME':'first',
                                                                        'DEST_CITY_NAME': 'first',
                                                                        'DEP_DELAY': np.sum,
                                                                        'ARR_DELAY': np.sum,
                                                                        'AIR_TIME': np.sum,
                                                                        'DISTANCE': np.sum,
                                                                        'OCCUPANCY_RATE':np.sum,
                                                                        'CANCELLED':'count'}).r

In [158]:
flights1xx = flights1.iloc[0:1000].sort_values('FL_DATE').reset_index(drop=True)

# Track used trips
used_indices = set()
round_trip_counter = {}

# Iterate to find round trip pairs
for i, row in flights1xx.iterrows():
    if i in used_indices:
        continue

    origin, destination, date = row['ORIGIN'], row['DESTINATION'], row['FL_DATE']

    # Look for a matching return trip: destination → origin, date > original
    mask = (
        (flights1xx['ORIGIN'] == destination) &
        (flights1xx['DESTINATION'] == origin) &
        (flights1xx['FL_DATE'] > date)
    )

    candidates = flights1xx[mask]
    for j, return_row in candidates.iterrows():
        if j not in used_indices:
            # Found valid round trip
            route_label = f"{origin} -> {destination} -> {origin}"
            round_trip_counter[route_label] = round_trip_counter.get(route_label, 0) + 1

            # Mark both legs as used
            used_indices.update({i, j})
            break

# Display results
for route, count in round_trip_counter.items():
    print(f"{route}: {count}")


STL -> BOS -> STL: 2
STL -> BNA -> STL: 1
STL -> BDL -> STL: 1
STL -> AUS -> STL: 1
STL -> ATL -> STL: 2
SMF -> BUR -> SMF: 4
SMF -> BOI -> SMF: 1
TPA -> BUF -> TPA: 2
TPA -> BNA -> TPA: 4
TPA -> BHM -> TPA: 1
TPA -> BDL -> TPA: 2
TPA -> AUS -> TPA: 1
TPA -> ATL -> TPA: 3
TPA -> ALB -> TPA: 1
SAT -> BNA -> SAT: 1
SAT -> ATL -> SAT: 1
SAN -> BNA -> SAN: 1
SAN -> AUS -> SAN: 1
SAN -> ATL -> SAN: 1
SAN -> ABQ -> SAN: 1
SAN -> BOI -> SAN: 1
SJC -> BUR -> SJC: 5
SJC -> BOI -> SJC: 1
SJC -> AUS -> SJC: 1
SFO -> BUR -> SFO: 2


In [174]:
flights1xx = flights1.iloc[0:10000].sort_values('FL_DATE').reset_index(drop=True)


# Create a trip column
flights1xx['trip'] = (flights1xx['ORIGIN'].astype(str)) + '_' + (flights1xx['DESTINATION'].astype(str))
flights1xx['reverse'] = (flights1xx['DESTINATION'].astype(str)) + '_' + (flights1xx['ORIGIN'].astype(str))

used_indices = set()
round_trip_counts = {}

# Iterate to find round trips
for i, row in flights1xx.iterrows():
    if i in used_indices:
        continue

    trip = row['trip']
    reverse = row['reverse']
    
    # Find a matching reverse trip not already used
    match = flights1xx[(flights1xx['trip'] == reverse) & (~flights1xx.index.isin(used_indices))]

    if not match.empty:
        j = match.index[0]
        route = f"{row['ORIGIN']} -> {row['DESTINATION']} -> {row['ORIGIN']}"
        round_trip_counts[route] = round_trip_counts.get(route, 0) + 1
        used_indices.update({i, j})

# Convert result to DataFrame
round_trip_df = pd.DataFrame(round_trip_counts.items(), columns=['route', 'count'])

print(round_trip_df)

                  route  count
0     SNA -> SJC -> SNA     12
1     SNA -> PHX -> SNA     13
2     SNA -> OAK -> SNA     12
3     SNA -> LAS -> SNA     10
4     SNA -> HOU -> SNA      3
...                 ...    ...
997   ATL -> DEN -> ATL      3
998   ATL -> DCA -> ATL      4
999   BOS -> IND -> BOS      1
1000  BDL -> DEN -> BDL      1
1001  BNA -> DCA -> BNA      2

[1002 rows x 2 columns]


In [170]:
import pandas as pd

# Sample data
df = pd.DataFrame({
    'origin': ['JFK', 'ORD', 'ORD', 'JFC', 'LAX', 'JFK', 'ORD'],
    'destination': ['ORD', 'JFK', 'JFC', 'ORD', 'SEA', 'ORD', 'JFK']
})

# Create a trip column
df['trip'] = df['origin'] + '_' + df['destination']
df['reverse'] = df['destination'] + '_' + df['origin']

used_indices = set()
round_trip_counts = {}

# Iterate to find round trips
for i, row in df.iterrows():
    if i in used_indices:
        continue

    trip = row['trip']
    reverse = row['reverse']
    
    # Find a matching reverse trip not already used
    match = df[(df['trip'] == reverse) & (~df.index.isin(used_indices))]

    if not match.empty:
        j = match.index[0]
        route = f"{row['origin']} -> {row['destination']} -> {row['origin']}"
        round_trip_counts[route] = round_trip_counts.get(route, 0) + 1
        used_indices.update({i, j})

# Convert result to DataFrame
round_trip_df = pd.DataFrame(round_trip_counts.items(), columns=['route', 'count'])

print(round_trip_df)

               route  count
0  JFK -> ORD -> JFK      2
1  ORD -> JFC -> ORD      1
